In [2]:
import csv
import pprint
import matplotlib.pyplot as plt
import re


#Legg ved path til nedlastet .csv fil fra nettleverandør
file_path = r'...'

#leser fil, overkomplisert og smått uleselig pga nettleverandør format
with open(file_path, 'r', encoding='utf-8') as file:
    csv_reader = csv.reader(file, delimiter=',', quotechar='"')
    pattern = re.compile(r',(?=(?:[^"]*"[^"]*")*[^"]*$)')
    
    topptekst = next(file).strip().split(',')
    #teller verdi endres om du vil begrense søk
    teller = 0
    data = {}
    gjennomsnitt = {}
    
    for row in csv_reader:
        items = pattern.split(row[0])
        items[2] = items[2].replace('"', '').replace(',', '.')
        
        date1, time1 = items[0].split(' ')
        date2, time2 = items[1].split(' ')
        
        
        value = float(items[2])
        
        if date1 in data:
            data[date1][time1] = value
        else:
            data[date1] = {time1: value}
        if date2 in data:
            data[date2][time2] = value
        else:
            data[date2] = {time2: value}
        
        teller += 1
        
    for date, times in data.items():
        total = sum(times.values())
        average = total / len(times)
        gjennomsnitt[date] = average
dates = list(gjennomsnitt.keys())
values = list(gjennomsnitt.values())

#illustrere gjennomsnitt kwh forbruk/time/dag via graf
plt.figure(figsize=(10, 5))  
plt.plot(dates, values, marker='o')
plt.xticks(rotation=45)
plt.title('Gjennomsnitt kwh/dag')
plt.xlabel('Dato')
plt.ylabel('Gjennomsnitt kwh/t')
plt.grid(True)
plt.show()

#registrere kwh forbruk gjennomsnitt /time/total
hourly_values = {f"{hour:02d}:00": [] for hour in range(24)}

for date, times in data.items():
    for time, value in times.items():
        hourly_values[time].append(value)


hourly_gjennomsnitt = {hour: sum(values) / len(values) for hour, values in hourly_values.items()}

 
hours = list(hourly_gjennomsnitt.keys())
values = list(hourly_gjennomsnitt.values())


#illustrere gjennomsnitt /time/total histogram
plt.figure(figsize=(10, 5))
plt.bar(hours, values)
plt.xticks(rotation=45)
plt.title('Gjennomsnitt kwh/time tidsperiode')
plt.xlabel('Tidspunkt')
plt.ylabel('Gjennomsnitt kwh/time')
plt.grid(True)
plt.show()

import numpy as np

#Her lages en plot over median time til time i tidsperiode, logger alle tilfeller over median ifh ønsket verdi
def plot_exceeding_values(hourly_values):
    # median for time til time
    hourly_medians = {hour: np.median(values) for hour, values in hourly_values.items()}
    hourly_thresholds = {hour: 1.5 * median for hour, median in hourly_medians.items()}

    # Sett en øvre grense for avvik fra median, for å finne antall overksridende dager
    exceeding_values = {hour: [value for value in values if value > threshold] 
                        for hour, values, threshold in zip(hourly_values.keys(), hourly_values.values(), hourly_thresholds.values())}

    for hour, values in exceeding_values.items():
        if values:  
            plt.scatter([hour]*len(values), values)  
            median = round(hourly_medians[hour], 2)  # Avrund til 2 desimaler for lesbarhet
            #en liste over medianer, nyttig men litt rotete
            print(f"Median for {hour}: {median}")  
    plt.xticks(rotation=45)  
    plt.title('Verdier som overskrider median med 50%')  
    plt.xlabel('Time i døgnet')
    plt.ylabel('Verdi i kwh, ikke i henhold til median')  
    plt.grid(True)
    plt.show()


plot_exceeding_values(hourly_values)


#Hvis du vil skrive ut all data i lesbart format
#    pprint.pprint(data)



PermissionError: [Errno 13] Permission denied: '...'